<a href="https://colab.research.google.com/github/SamuelOnyangoOmondi/alu-machine_learning/blob/main/Credit_Score_Prediction_7b21c9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'credit-score-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2289007%2F3846912%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T091322Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4d44f033c0c9ba0151c8ca9765724c2c5e4b31989b593883a3a16601518dc3a82a14cc6806a63bb8c3f5284081848ba594d02e3aaebba92b8aa1f5044cf15b33763674020a1f3e978e7babdf640af2c192470b18bb03ed7b75cdb34638aa4f02b3037ee558984afc8d53c106911f0f49b2fe3c296aa1764237e98ea0b6ff0e1bfdb238c7a81fa26dff12bc73d3bf27419cffbcee1e64d35d04bc7279507456f49121a2632a41949eaaf01da8ffa625239413a4be04cae6b8b0fb28cb13e76ed66fa87a8f31f765462c4158b234ffbe2a5d9d9e3c9d34cd142b3a17a7b066309305675990667ed86b36b04b02cabd2d87fae10652c40cc8433e8d7b8d01544f77'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 9973595 bytes downloaded
Downloaded and uncompressed: credit-score-classification
Data source import complete.


# Predicting Credit Scores
Welcome to this data science project focused on predicting credit scores in the banking industry. In this notebook, we embark on a mission to harness the power of data and machine learning to address a crucial aspect of financial decision-making.

### Mission Objective:
Our primary goal is to develop a robust and accurate predictive model for credit scores. Credit scores play a pivotal role in the world of finance, influencing lending decisions, interest rates, and access to various financial services. By predicting credit scores effectively, we aim to:

* Empower Informed Decisions: Provide financial institutions with a tool to make informed decisions regarding loan approvals, interest rates, and credit limits, ultimately improving risk management.

* Enhance Financial Inclusion: Enable fairer access to credit by identifying individuals who may be creditworthy but are often underserved by traditional credit scoring methods.

* Minimize Defaults: Reduce the risk of defaults by identifying high-risk applicants early in the lending process.

## DATA OVERVIEW

'id': A unique identifier for each data record.

'customer_id': An identifier for each customer, allowing you to associate multiple records with the same individual.

'month': The month of the data record, indicating when the data was collected or relevant.

'name': The name of the customer, which may be used for identification purposes.

'age': The age of the customer, providing information about their age demographic.

'ssn': The Social Security Number (SSN) of the customer, a unique identifier used for verification.

'occupation': The occupation or profession of the customer, which can help understand their employment status.

'annual_income': The annual income of the customer, a crucial financial parameter.

'monthly_inhand_salary': The monthly salary or income available to the customer after deductions.

'num_bank_accounts': The number of bank accounts held by the customer, indicating their banking activity.

'num_credit_card': The number of credit cards held by the customer, reflecting their credit usage.

'interest_rate': The interest rate associated with the customer's financial products, such as loans or credit cards.

'num_of_loan': The number of loans the customer has, providing insight into their debt obligations.

'type_of_loan': The types of loans the customer holds, which can include mortgages, personal loans, etc.

'delay_from_due_date': The delay in payment from the due date for loans or credit cards, indicating their payment behavior.

'num_of_delayed_payment': The number of delayed payments made by the customer.

'changed_credit_limit': Changes in the customer's credit limit, which can affect their credit utilization.

'num_credit_inquiries': The number of credit inquiries made by the customer, potentially affecting their credit score.

'credit_mix': The composition of the customer's credit accounts, which can impact their credit profile.

'outstanding_debt': The amount of outstanding debt owed by the customer.

'credit_utilization_ratio': The ratio of credit used to the total available credit, a key factor in credit scoring.

'credit_history_age': The age of the customer's credit history, influencing their creditworthiness.

'payment_of_min_amount': How customers handle the minimum payment amount on credit cards or loans.

'total_emi_per_month': The total Equated Monthly Installment (EMI) payments made by the customer.

'amount_invested_monthly': The amount the customer invests on a monthly basis, if applicable.

'payment_behaviour': The behavior of the customer regarding their payments, reflecting their financial responsibility.

'monthly_balance': The monthly balance in the customer's financial accounts.

'credit_score': The target variable representing the customer's credit score, which we aim to predict.

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('/kaggle/input/credit-score-classification/train.csv')

In [ ]:
df = data.copy()

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
pd.set_option('display.max_columns', None)
df.sample(5)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

## INVESTIGATION COLUMN BY COLUMN

In [ ]:
object_columns = df.columns.tolist()

In [ ]:
for col in object_columns:
    print('Column Name: '+col)
    print("**"*20)
    print(df[col].value_counts(dropna=False))
    print('END', "--"*18, '\n')

## Categorical Features Handling Plan


##### month
* int64 --> str
* This column will be encoded

##### occupation
* `_______` values will be deleted.
* Values will be imputed using the mode, which represents the most common value associated with each unique Customer_ID
* This column will be encoded.

#####  type_of_loan
* Get every specific loan type.
* Binary encoding. If a person has that specific loan type in the cell, it will be True otherwise False.
* Implement mode imputer to missing values

##### credit_mix
* `_` values will be deleted
* Missing values will be imputed using the mode, which represents the most common value associated with each unique Customer_ID
* This column will be encoded.

##### payment_of_min_amount
* `NM` values will be deleted
* Missing values will be imputed using the mode, which represents the most common value associated with each unique Customer_ID
* This column will be encoded.

##### payment_behaviour
* `!@9#%8` values will be deleted.
* Missing values will be imputed using the mode, which represents the most common value associated with each unique Customer_ID
* This column will be encoded.

##### credit_score
* This column is the target. There are three values; Standard, Poor, Good. They will be replaced as Poor --> 0, Good --> 1, Standard --> 2

##### name / ssn / customer_id
* These columns will be deleted at the end of the categorical feature handling process.

In [ ]:
categorical_columns = ['credit_mix',
                      'month',
                      'occupation',
                      'payment_behaviour',
                      'payment_of_min_amount',
                      'type_of_loan',
                      'credit_score',
                      'id',
                      'customer_id',
                      'name',
                      'ssn']

In [ ]:
class CategoricalTransformer():
    """
    This class provides methods for handling and transforming categorical features in a DataFrame.

    Parameters:
    - input_df (pd.DataFrame): The input DataFrame containing categorical features.

    Methods:
    - month_handling: Encodes and handles the 'month' column.
    - occupation_handling: Encodes and handles the 'occupation' column.
    - type_of_loan_handling: Handles and encodes the 'type_of_loan' column.
    - credit_mix_handling: Encodes and handles the 'credit_mix' column.
    - payment_of_min_amount_handling: Encodes and handles the 'payment_of_min_amount' column.
    - payment_behaviour_handling: Encodes and handles the 'payment_behaviour' column.
    - credit_score_handling: Encodes the 'credit_score' column.
    - column_deleter: Deletes unnecessary columns and returns the modified DataFrame.

    """
    def __init__(self,input_df:pd.DataFrame):
        self.df = input_df

    def month_handling(self) -> pd.DataFrame:
        if 'month' not in self.df.columns:
            raise ValueError ('"month" not found in the DataFrame.')


        self.df['month'] = self.df['month'].astype(str)
        # Encoding without first value
        month_dummies = pd.get_dummies(self.df['month'], prefix='month', drop_first=True, dtype=int)

        # Creating new dataframe with dummies (month_january) and without original column.
        self.df = pd.concat([self.df, month_dummies], axis=1)
        self.df.drop('month', axis=1, inplace=True)

        return self.df

    def occupation_handling(self) -> pd.DataFrame:
        if 'occupation' not in self.df.columns:
            raise ValueError ('"occupation" not found in the DataFrame.')

        # Deleting strange values
        self.df['occupation'] = self.df['occupation'].replace("_______", pd.NA)

        # Replacing missing values in each row, associated with a specific customer_id, with the most frequent value.
        self.df['occupation'].astype(str)
        self.df['occupation'] = self.df.groupby('customer_id')['occupation'].transform(lambda x: x.mode()[0])

        # Binary encoding without first value.
        occupation_dummies = pd.get_dummies(self.df['occupation'], drop_first=True, prefix='occupation', dtype=int)
        self.df = pd.concat([self.df, occupation_dummies], axis=1)
        self.df.drop('occupation', axis=1, inplace=True)

        return self.df

    def type_of_loan_handling(self) -> pd.DataFrame:
        '''
        There are values like "Credit-Builder Loan, and Mortgage Loan". Firs of all, this method extracts all
        specific type of loans (Credit-Builder Loan,Mortgage Loan etc.). Then creates a columns like; "has_creadit_builder_loan".
        While creating these columns it checks if that specific loan type exist in the row then return true if exist or false
        if not. Finally, true and false values return zeros and ones and original column deleted.
        '''

        if 'type_of_loan' not in self.df.columns:
            raise ValueError ('type_of_loan not found in the DataFrame.')

        loan_type_split = self.df['type_of_loan'].str.split(r', and |, | and |,').dropna()

        # Flatten the resulting list of lists
        loan_types_list = [item.strip() for sublist in loan_type_split.tolist() for item in sublist]

        # Geting the unique loan types
        unique_loan_types = set(loan_types_list)


        for loan_type in unique_loan_types:
            if pd.notna(loan_type):
                # Creating new columns without spaces but instead with "_"
                new_column = (loan_type.replace(" ", "_")).lower()

                # This line; created columns like "has_student_loan" and return true or false, if it's emtpy return pd.NA
                self.df[f'has_{new_column}'] = self.df['type_of_loan'].apply(lambda x: loan_type in x if pd.notna(x) else pd.NA)
                #This line; transforms true and false values into zeros and ones
                self.df[f'has_{new_column}'] = self.df[f'has_{new_column}'].apply(lambda x: int(x) if x is not pd.NA else x)

                # Filling null values with mode. Since these columns are binary, mode imputation has used to fill null values.
                mode_value = self.df[f'has_{new_column}'].mode().iloc[0]
                self.df[f'has_{new_column}'].fillna(mode_value, inplace=True)

        self.df.drop('type_of_loan', axis=1, inplace=True)

        return self.df

    def credit_mix_handling(self) -> pd.DataFrame:
        if 'credit_mix' not in self.df.columns:
            raise ValueError ('"credit_mix" not found in the DataFrame.')

        # Deleting strange values like "_"
        self.df['credit_mix'] = self.df['credit_mix'].replace("_", pd.NA)

        # Replacing missing values in each row, associated with a specific customer_id, with the most frequent value.
        self.df['credit_mix']= self.df.groupby('customer_id')['credit_mix'].transform(lambda x: x.mode()[0])
        self.df['credit_mix'].astype(str)

        # Binary encoding
        credit_mix_dummies = pd.get_dummies(self.df['credit_mix'], drop_first=True, prefix='credit_mix', dtype=int)
        self.df = pd.concat([self.df, credit_mix_dummies], axis=1)
        self.df.drop('credit_mix', axis=1, inplace=True)

        return self.df

    def payment_of_min_amount_handling(self) -> pd.DataFrame:
        if 'payment_of_min_amount' not in self.df.columns:
            raise ValueError ('"payment_of_min_amount" not found in the DataFrame.')

        # Deleting strange values like "NM"
        self.df['payment_of_min_amount'] = self.df['payment_of_min_amount'].replace("NM", pd.NA)

        # Replacing missing values in each row, associated with a specific customer_id, with the most frequent value.
        self.df['payment_of_min_amount']= self.df.groupby('customer_id')['payment_of_min_amount'].transform(lambda x: x.mode()[0])
        self.df['payment_of_min_amount'].astype(str)

        # Binary encoding
        payment_of_min_amount_dummies = pd.get_dummies(self.df['payment_of_min_amount'],drop_first=True ,prefix='payment_of_min_amount', dtype=int)
        self.df = pd.concat([self.df, payment_of_min_amount_dummies], axis=1)
        self.df.drop('payment_of_min_amount', axis=1, inplace=True)

        return self.df

    def payment_behaviour_handling(self) -> pd.DataFrame:
        if 'payment_behaviour' not in self.df.columns:
            raise ValueError ('"payment_behaviour" not found in the DataFrame.')

        # Deleting strange values like "!@9#%8 "
        self.df['payment_behaviour'] = self.df['payment_behaviour'].replace("!@9#%8", pd.NA)

        # Replacing missing values in each row, associated with a specific customer_id, with the most frequent value.
        self.df['payment_behaviour']= self.df.groupby('customer_id')['payment_behaviour'].transform(lambda x: x.mode()[0])
        self.df['payment_behaviour'].astype(str)

        # Binary encoding
        payment_behaviour_dummies = pd.get_dummies(self.df['payment_behaviour'],drop_first=True , dtype=int)
        self.df = pd.concat([self.df, payment_behaviour_dummies], axis=1)
        self.df.drop('payment_behaviour', axis=1, inplace=True)

        return self.df


    def credit_score_handling(self) -> pd.DataFrame:
        if 'credit_score' not in self.df.columns:
            raise ValueError ('"credit_score" not found in the DataFrame.')

        # After executing this method values will be Poor==0, Good==1, Standard==2
        mapping = {"Standard":2, "Poor":0, "Good":1}
        self.df['credit_score'] = self.df['credit_score'].replace(mapping)

        # Since this column is the target variable, I expect it to be located at the end of the dataset.
        credit_score_column = self.df['credit_score']
        self.df = self.df.drop(columns=['credit_score'])
        self.df['credit_score'] = credit_score_column

        return self.df

    #THIS COLUMN SHOULD RUN IN THE END
    def column_deleter(self) -> pd.DataFrame:

        # All columns lowered one more time.
        self.df.columns = self.df.columns.str.lower()

        # Deleting unnecessary features.
        columns_to_drop = ['name', 'ssn', 'customer_id', 'id']
        self.df.drop(columns=columns_to_drop, inplace=True)


        return self.df




## Numeric Features Handling Plan

delete all negative numbers

### INTEGERS
##### age / num_bank_accounts / num_credit_card /  interest_rate / num_of_loan / delay_from_due_date / num_of_delayed_payment / num_credit_inquiries /
* There are values like; 24_. Numerics should be parsed from others. // Non number characters should be deleted. // Datapoints should contain only numeric characters.
* Replacing missing values in each row, associated with a specific customer_id, with the most frequent value. --> INT

##### credit_history_age
* From "2 Years 4 Months" format to 28 --> INT

### FLOATS

#####  annual_income / monthly_inhand_salary / changed_credit_limit( _ ) / outstanding_debt / total_emi_per_month /
* Non number characters should be deleted. // Datapoints should contain only numeric characters.
* Replacing missing values in each row, associated with a specific customer_id, with the most frequent value. --> FLOAT .4

##### amount_invested_monthly	/ monthly_balance
* `__10000__` and `__-333333333333333333333333333__` values will be deleted if there any.
* Non number characters should be deleted. // Datapoints should contain only numeric characters.
* Replacing missing values with the mean --> FLOAT .4




In [ ]:
integer_columns_to_handle = ['age',
                            'num_bank_accounts',
                            'num_credit_card',
                            'interest_rate',
                            'num_of_loan',
                            'delay_from_due_date',
                            'num_of_delayed_payment',
                            'num_credit_inquiries',
                            'credit_history_age']

float_columns_to_handle = ['credit_utilization_ratio',
                           'annual_income',
                           'monthly_inhand_salary',
                           'changed_credit_limit',
                           'outstanding_debt',
                           'total_emi_per_month',
                           'amount_invested_monthly',
                           'monthly_balance']

In [ ]:
class NumericTransformer():
    '''
    This class handles numeric features in a DataFrame. It provides methods for handling floating-point features,
    integer features, and transforming the 'credit_history_age' column.

    Parameters:
    - input_df (pd.DataFrame): The input DataFrame to be processed.

    Methods:
    - floats_handling: Handles floating-point features.
    - integers_handling: Handles integer features.
    - transform_credit_history_age: Transforms the 'credit_history_age' column.

    '''
    def __init__(self, input_df:pd.DataFrame):
        '''
        Initializes the NumericTransformer with the input DataFrame.

        Parameters:
        - input_df (pd.DataFrame): The input DataFrame to be processed.
        '''
        self.df = input_df
        self.df.columns = self.df.columns.str.lower()

    def floats_handling(self) -> pd.DataFrame:
        '''
        Handles floating-point features in the DataFrame.

        Returns:
        - pd.DataFrame: The DataFrame with processed floating-point features.
        '''
        columns_to_process = ['credit_utilization_ratio',
                           'annual_income',
                           'monthly_inhand_salary',
                           'changed_credit_limit',
                           'outstanding_debt',
                           'total_emi_per_month',
                           'amount_invested_monthly',
                           'monthly_balance']


        # First check if the specified columns exist in dataframe
        for column in columns_to_process:
            if column not in self.df.columns:
                raise ValueError(f"Column '{column}' not found in the DataFrame.")

        # Deleting specific strange values
        mapping = {'__-333333333333333333333333333__':pd.NA,
                  '__10000__':pd.NA}
        self.df.replace(mapping, inplace=True)

        # Cleaning the datapoint from non-numeric characters.
        pattern = r'[^0-9.]' # Keeping numbers 0 to 9 and dots(".") because these are floats.
        for column in columns_to_process:
            self.df[column] = self.df[column].astype(str) # All columns will be changed as str to impelement replace
            self.df[column] = self.df[column].str.replace(pattern, '', regex=True) # All characters gone except 0 to 9 and dot
            self.df[column] = self.df[column].replace('', pd.NA) # After deleting characters some rows could be emtpy, they're NA now


        ## FILLING MISSING ROWS ##
        # Mode imputation will be applied to some features and mean will be applied to others(last 2)

        # Imputing columns except first and last three.
        for mode_column in columns_to_process[1:-3]:
            self.df[mode_column] = self.df.groupby('customer_id')[mode_column].transform(lambda x: x.mode()[0])


        ### columns_to_process[5]/'total_emi_per_month' has 0 values so it will be treated separately ###
        # Converting float to be able to apply imputations.
        self.df['total_emi_per_month']= (self.df['total_emi_per_month'].astype(float)).round(4)

        # Implementing mode imputation to non-null values.
        self.df['total_emi_per_month'] = self.df.groupby('customer_id')['total_emi_per_month'].transform(lambda x: x.mode()[0] if not x.isnull().all() else pd.NA)

        # There are so many zeros and they're not valid. Replacing(deleting) zeros with pd.NA then mean imputing to missing rows.
        self.df['total_emi_per_month'] = self.df['total_emi_per_month'].replace(0, pd.NA)
        self.df['total_emi_per_month'].fillna(self.df['total_emi_per_month'].median(), inplace=True)



        ## Imputation of last two features; amount_invested_monthly, monthly_balance ##
        # There will be mean imputation for these two column
        # total_emi_per_month =  Skewness: 2.55, Kurtosis: 8.74
        # monthly_balance = Skewness: 1.59, Kurtosis: 2.95
        for mean_column in columns_to_process[-2:]:
            # Converting the column to a numeric data type, handling non-numeric values
            self.df[mean_column] = pd.to_numeric(self.df[mean_column], errors='coerce')
            # Filling missing values with the mean
            self.df[mean_column] = self.df[mean_column].fillna(self.df[mean_column].mean())

        # All columns will be float data type.
        self.df[columns_to_process] = self.df[columns_to_process].astype(float).round(4)

        return self.df


    def integers_handling(self) -> pd.DataFrame:
        '''
        Handles integer features in the DataFrame.

        Returns:
        - pd.DataFrame: The DataFrame with processed integer features.
        '''
        features_to_process = ['age',
                              'num_credit_card',
                              'interest_rate',
                              'delay_from_due_date',
                              'num_of_delayed_payment',
                              'num_of_loan',
                              'num_bank_accounts',
                              'num_credit_inquiries'
                              ]


        # First check if the specified columns exist in dataframe
        for column in features_to_process:
            if column not in self.df.columns:
                raise ValueError(f"Column '{column}' not found in the DataFrame.")



        pattern = r'[^0-9]'
        for column in features_to_process:
            self.df[column] = self.df[column].astype(str) # All columns will be changed as str to ne able to impelement replace
            self.df[column] = self.df[column].str.replace(pattern, '', regex=True) # All characters gone except 0 to 9
            self.df[column] = self.df[column].replace('', pd.NA) #After deleting characters some rows could be emtpy, they're NA now

            # Replacing missing values in each row, associated with a specific customer_id, with the most frequent value.
            self.df[column] = self.df.groupby('customer_id')[column].transform(lambda x: x.mode()[0])
            # Tranformed as int.
            self.df[column] = self.df[column].astype(int)


        #So many invalid zeros, they need to be handled (last three values in features_to_process).
        for column in features_to_process[-3:]:

            # I dont remember why I implemented this line of code twice but I am gonna keep it :)
            self.df[column] = self.df.groupby('customer_id')[column].transform(lambda x: x.mode()[0] if not x.isnull().all() else pd.NA)
            # Replacing zeros with pd.NA then implementing mean imputation.
            self.df[column] = self.df[column].replace(0, pd.NA)
            self.df[column].fillna(self.df[column].median(), inplace=True)
            self.df[column] = self.df[column].astype(int)


        return self.df



    def transform_credit_history_age(self) -> pd.DataFrame:
        '''
        Transforms the 'credit_history_age' column from textual format to integer format
        (e.g., 3 Years and 1 Month to 37)

        Returns:
        - pd.DataFrame: The DataFrame with the transformed 'credit_history_age' column.
        '''
        column_name = 'credit_history_age'

        # There are few kinds of null types so I replaced all of them with pd.NA
        self.df[column_name] = self.df[column_name].fillna(pd.NA)

        # Replacing missing values in each row, associated with a specific customer_id, with the most frequent value.
        self.df[column_name] = self.df.groupby('customer_id')[column_name].transform(lambda x: x.mode()[0] if not x.isnull().all() else pd.NA)

        # Converting the column to string type
        self.df[column_name] = self.df[column_name].astype(str)

        # Extracting the numeric values using regular expressions
        extracted_values = self.df[column_name].str.extract(r'(\d+) Years and (\d+) Months')

        # Converting the extracted values to integers and perform the calculation.
        self.df[column_name] = (extracted_values[0].astype(int) * 12) + extracted_values[1].astype(int)

        return self.df



## Creating the preprocessor

In [ ]:
class Preprocessor():
    '''
    A utility class to perform transformations on a DataFrame.
    '''
    def __init__(self, dataframe:pd.DataFrame):
        '''
        Initializes numeric and categorical transformes.

        Paramaters:
        - dataframe(pd.DataFrame): The input DataFrame to be transformed
        '''

        # Creating copy of original dataframe
        self.df = dataframe.copy()

        # Creating instances of transformers.
        self.num_handler = NumericTransformer(input_df=self.df)
        self.cat_handler = CategoricalTransformer(input_df=self.df)


    def transform(self) -> pd.DataFrame:

        # Numeric transformation
        self.df = self.num_handler.floats_handling()
        self.df = self.num_handler.integers_handling()
        self.df = self.num_handler.transform_credit_history_age()

        # Categorical transformation
        self.df = self.cat_handler.credit_mix_handling()
        self.df = self.cat_handler.month_handling()
        self.df = self.cat_handler.occupation_handling()
        self.df = self.cat_handler.payment_behaviour_handling()
        self.df = self.cat_handler.payment_of_min_amount_handling()
        self.df = self.cat_handler.type_of_loan_handling()
        self.df = self.cat_handler.credit_score_handling()
        self.df = self.cat_handler.column_deleter()

        return self.df


### Check if there is any problem about transformation

In [ ]:
preprocessor = Preprocessor(dataframe=df)

In [ ]:
transformed_df = preprocessor.transform() # Creating a new transformed dataframe

In [ ]:
transformed_df.sample(10)

In [ ]:
transformed_df.info()

##### It seems there is no problem about null values or data types.

### NUMERIC COLUMNS HISTOGRAMS (transformed but not standardized)

In [ ]:
numeric_columns = integer_columns_to_handle + float_columns_to_handle

In [ ]:
import seaborn as sns
import matplotlib.pyplot as  plt
%matplotlib inline
from scipy.stats import skew, kurtosis

In [ ]:
# Examining distribution of the numeric columns with histograms

n_rows = 5
n_cols = 4

# Creating subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 12))

# Flatten the axes array to iterate over all subplots
axes = axes.flatten()

# Loop through the features and create histplots
for i, feature in enumerate(numeric_columns):
    sns.histplot(data=transformed_df, x=feature, bins=30, ax=axes[i])

    axes[i].set_title(f'Distribution of {feature}')  # Replace with your title

# Hide any remaining empty subplots
for j in range(i + 1, n_rows * n_cols):
    fig.delaxes(axes[j])

plt.tight_layout()  # Ensure proper spacing
plt.show()

## Feature Analyzing

In [ ]:
class FeatureAnalyzer():
    '''
    This class provides methods to analyze numerical columns in a DataFrame, including statistics such as mean,
    median, standard deviation, variance, kurtosis, skewness, maximum, minimum, and count of zeros.

    Args:
        input_df (pd.DataFrame): The DataFrame containing the numerical columns to be analyzed.

    Methods:
        column_statistics(columns): Calculates statistics for the specified columns.

    '''
    def __init__(self, input_df):
        self.df = input_df

    def column_statistics(self, columns):
        '''
        Calculate statistics for the specified columns.

        Args:
            columns (list of str): A list of column names to analyze.

        Returns:
            pd.DataFrame: A DataFrame containing statistics for the specified columns.

        Example:
            analyzer = FeatureAnalyzer(data_frame)
            stats = analyzer.column_statistics(['age', 'income'])
        '''

        invalid_columns = [col for col in columns if col not in self.df.columns]
        if invalid_columns:
            print("Invalid columns:", invalid_columns)
            return

        stats_dict = {}

        for column in columns:
            # Replace non-numeric values with NaN
            numeric_values = pd.to_numeric(self.df[column], errors='coerce')

            count_of_zeros = (numeric_values == 0).sum()
            std = numeric_values.std()
            variance = numeric_values.var()
            mean = numeric_values.mean()
            median = numeric_values.median()
            kurt = kurtosis(numeric_values).round(2)
            skewness = skew(numeric_values).round(2)
            maximum = numeric_values.max()
            minumum = numeric_values.min()

            stats_dict[column] = {
                'Count of Zeros': count_of_zeros,
                'Standard Deviation': std,
                'Variance': variance,
                'Mean': mean,
                'Median': median,
                'Kurtosis': kurt,
                'Skewness': skewness,
                'Max Value': maximum,
                'Min Value': minumum
            }

        return pd.DataFrame.from_dict(stats_dict, orient='index')

### Before NumericTransformer implemented.

In [ ]:
analyzer1 = FeatureAnalyzer(input_df=df)

In [ ]:
analyzer1.column_statistics(numeric_columns)

### NumericTransformer implemented (no standardization)

In [ ]:
analyzer2 = FeatureAnalyzer(input_df=transformed_df)

In [ ]:
analyzer2.column_statistics(numeric_columns)

##### All columns seem fine for me. There are extreme values but let's see the model performance.

# MODELING

In [ ]:
#from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
#from sklearn.tree import DecisionTreeClassifier
#from catboost import CatBoostClassifier
#from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [ ]:
X = transformed_df.drop('credit_score', axis=1)
y = transformed_df['credit_score']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer, f1_score

In [ ]:
models = {
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier()
}

In [ ]:
params = {
    "Random Forest": {
        'n_estimators': [16, 32, 64, 128],
        'max_depth': [None, 10, 20],
       # 'min_samples_split': [5, 10],
        #'min_samples_leaf': [2, 5],
        #'bootstrap': [True, False],
    },
    "XGBoost": {
        'learning_rate': [0.1, 0.05, 0.001],
        'n_estimators': [16, 32, 64, 128],
        #'max_depth': [3, 4, 5],
        'subsample': [0.7, 0.8]
    }
}


In [ ]:
f1_scorer = make_scorer(f1_score, average='weighted')

In [ ]:
grid_searches = {}
for model_name, model in models.items():
    grid_search = GridSearchCV(
        model,
        params[model_name],
        cv=3,  # Use the number of desired cross-validation folds
        scoring=f1_scorer,
        n_jobs=-1,  # Use all available CPU cores
        verbose=2,
    )
    grid_searches[model_name] = grid_search


In [ ]:
grid_searches

In [ ]:
best_models = {}
for model_name, grid_search in grid_searches.items():
    grid_search.fit(X_train, y_train)  # X_train and y_train are your training data
    best_models[model_name] = grid_search.best_estimator_


In [ ]:
best_models

In [ ]:
best_f1_score = -1  # Initialize with a low value
best_model = None

for model_name, grid_search in grid_searches.items():
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best F1-score for {model_name}: {grid_search.best_score_}")
    print("=="*25,"\n")

    if grid_search.best_score_ > best_f1_score:
        best_f1_score = grid_search.best_score_
        best_model = grid_search.best_estimator_

if best_model is not None:
    print("Best model based on F1-score:")
    print(best_model)
    print(f"Best F1-score: {best_f1_score}")

In [ ]:
print(best_model)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
y_pred = best_model.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Plot a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix for {best_model}')
plt.show()

# Print the classification report
print(f"Classification Report for {best_model}:\n")
print(report)

## This confusion matrix means that;

### Class 0 (Poor):
* Actual Count: 5874
* Correctly Predicted as Poor: 4973
* Incorrectly Predicted as Good: 4
* Incorrectly Predicted as Standard: 897

### Class 1 (Good):
* Actual Count: 3527
* Correctly Predicted as Good: 2689
* Incorrectly Predicted as Poor: 3
* Incorrectly Predicted as Standard: 835

### Class 2 (Standard):
* Actual Count: 10599
* Correctly Predicted as Standard: 8658
* Incorrectly Predicted as Poor: 1243
* Incorrectly Predicted as Good: 698

In [ ]:
'''
FOR THOSE WHO WANT TO FIND OUT IF THERE IS ANY CHANCE TO HAVE BETTER OPTIONS.
IF YOU WANT TO EXPLORE MORE JUST REPLACE models and params WITH REAL ONES.
'''

models = {
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(),
    "CatBoost": CatBoostClassifier(verbose=False),
    "AdaBoost": AdaBoostClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
}


params = {
    "Random Forest": {
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False],
    },
    "Decision Tree": {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    },
    "Gradient Boosting": {
        'learning_rate': [0.1, 0.01, 0.001],
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'max_depth': [3, 4, 5],
        'subsample': [0.6, 0.7, 0.8, 0.9],
    },
    "XGBoost": {
        'learning_rate': [0.1, 0.01, 0.05, 0.001],
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'max_depth': [3, 4, 5],
        'subsample': [0.6, 0.7, 0.8, 0.9],
    },
    "CatBoost": {
        'depth': [6, 8, 10],
        'learning_rate': [0.01, 0.05, 0.1],
        'iterations': [30, 50, 100],
    },
    "AdaBoost": {
        'learning_rate': [0.1, 0.01, 0.5, 0.001],
        'n_estimators': [8, 16, 32, 64, 128, 256],
    },
    "K-Nearest Neighbors": {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'p': [1, 2],
    },
}